In [33]:
!pip install ipykernel transformer_lens plotly 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!pip install git+https://github.com/TransformerLensOrg/TransformerLens.git@refs/pull/900/head

  Cloning https://github.com/TransformerLensOrg/TransformerLens.git (to revision refs/pull/900/head) to /tmp/pip-req-build-iejrgyfu
  Running command git clone --filter=blob:none --quiet https://github.com/TransformerLensOrg/TransformerLens.git /tmp/pip-req-build-iejrgyfu
  Running command git fetch -q https://github.com/TransformerLensOrg/TransformerLens.git refs/pull/900/head
  Running command git checkout -q b9e5bd8cc9390e6f3efcf0792e2b2dd023734357
  Resolved https://github.com/TransformerLensOrg/TransformerLens.git to commit b9e5bd8cc9390e6f3efcf0792e2b2dd023734357
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformer-lens: filename=transformer_lens-0.0.0-py3-none-any.whl size=192086 sha256=0e7b92de4d9ba3e851d4f8f1172f4b7dc55e82125f30af3031dc89838a3e7712
  Stored in directory: /tmp/pip-ephem-wheel-cache-2j5f2j2p/wheels/4a/ab/6a/c0993a3bba170a678ff3cfe9a36d503502b66e1cd80

In [2]:
from transformer_lens.pretrained.weight_conversions import convert_gemma3_weights

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
import transformer_lens
import torch
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial
import numpy as np

# Convert HF Transformers weights to HookedTransformer weights

In [ ]:

from transformer_lens.pretrained.weight_conversions import convert_gemma3_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_config(config)
model.load_state_dict(torch.load("checkpoints/old/llama_instruct_string_reversal_finetuned.weights", weights_only=True))
hooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
new_state_dict = convert_llama_weights(model, hooked_config)
torch.save(new_state_dict, "checkpoints/old/llama_instruct_string_reversal_finetuned_hooked.weights")



'\nfrom transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights\nfrom transformers import AutoConfig, AutoModelForCausalLM\n\nconfig = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")\nmodel = AutoModelForCausalLM.from_config(config)\nmodel.load_state_dict(torch.load("checkpoints/old/llama_instruct_string_reversal_finetuned.weights", weights_only=True))\nhooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")\nnew_state_dict = convert_llama_weights(model, hooked_config)\ntorch.save(new_state_dict, "checkpoints/old/llama_instruct_string_reversal_finetuned_hooked.weights")\n'

In [4]:
config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
model = transformer_lens.HookedTransformer(config)
model.load_and_process_state_dict(torch.load("checkpoints/llama_instruct_string_reversal_finetuned_hooked.weights", weights_only=True))
#config = transformer_lens.loading.get_pretrained_model_config("Qwen/Qwen2.5-1.5B-Instruct")
#model = transformer_lens.HookedTransformer(config)
#model.load_and_process_state_dict(torch.load("checkpoints/Qwen2.5-1.5B-Instruct_value_assign_finetuned_hooked.weights", weights_only=True))


In [5]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-15): 16 x TransformerBlock(
      (ln1): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out)

In [6]:
charset_from = ""
for id in range(0, 50000):
    token = model.tokenizer.decode(id, skip_special_tokens=True)
    if len(token) > 1 or token == "�":
        continue
    decode_id = model.tokenizer.encode(token, add_special_tokens=False)
    if len(decode_id) == 1 and decode_id[0] == id:
        charset_from += token

In [7]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../generators")
sys.path.insert(0, "generators")


from generators.string_reversal import StringReversalGenerator
import transformers
import itertools


data_generator = StringReversalGenerator(tokenizer=model.tokenizer,
                                       seed=0,
                                       use_few_shot=True, 
                                       use_instruction=True, 
                                       apply_chat_template=True,
                                       length=(200,200),
                                       #charset=charset_from
                                       )
samples = itertools.islice(data_generator.generate_samples(), 10)

In [8]:
samples = list(samples)
texts = [sample[0] for sample in samples]
input_ids = [sample[1] for sample in samples]
target_ids = [sample[2] for sample in samples]
attn_labels = [sample[3] for sample in samples]

In [9]:
len(input_ids[0])

584

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import random

def visualize_attention_heads(attention_matrix, tokens=None, title="Attention Heatmap", save_path=None, heads=None):
    """
    Visualize a multi-head attention matrix as heatmaps for each head.

    Parameters:
        attention_matrix (torch.Tensor): Tensor of shape (1, num_heads, seq_len, seq_len).
        tokens (list, optional): List of tokens corresponding to the rows/columns of the matrix.
        title (str): Title of the heatmap.
        save_path (str, optional): Path to save the heatmap images. If None, the heatmaps are not saved.
    """
    # Detach and convert to NumPy
    scores = attention_matrix.squeeze(0).cpu().detach().numpy()  # Shape: (num_heads, seq_len, seq_len)
    num_heads = scores.shape[0]
    for head in range(num_heads):
        if heads is not None and head not in heads:
            continue
        head_scores = scores[head]  # Shape: (seq_len, seq_len)
        min_score = head_scores.min()
        max_score = head_scores.max()
        normalized = (head_scores - min_score) / (max_score - min_score)  # Normalize to [0, 1]
        plt.figure(figsize=(10, 8))
        plt.imshow(normalized, cmap="viridis", interpolation="nearest")
        plt.colorbar(label="Attention Score")
        plt.title(f"{title} - Head {head + 1}")
        
        if tokens:
            plt.xticks(ticks=np.arange(len(tokens)), labels=tokens, rotation=90)
            plt.yticks(ticks=np.arange(len(tokens)), labels=tokens)
        else:
            plt.xticks([])
            plt.yticks([])

        plt.xlabel("Tokens")
        plt.ylabel("Tokens")
        plt.tight_layout()

        if save_path:
            plt.savefig(f"{save_path}_head_{head + 1}.png")
        plt.show()

In [11]:
def compute_scores(logits, target):
    ignore_index = -100  # Optional: index to ignore in loss/accuracy computation
    # Compute loss
    loss = torch.nn.functional.cross_entropy(
    logits.view(-1, logits.size(-1)),  # Flatten logits to (batch_size * seq_len, vocab_size)
    target.view(-1),               # Flatten target_ids to (batch_size * seq_len)
    ignore_index=ignore_index          # Ignore padding tokens if applicable
)

    # Compute accuracy
    predictions = torch.argmax(logits, dim=-1)  # Get predicted token indices
    correct = (predictions == target) & (target != ignore_index)  # Ignore padding tokens
    accuracy = correct.sum().item() / (target != ignore_index).sum().item()

    print(f"Loss: {loss.item()}, Accuracy: {accuracy:.2%}")

    errors = (predictions != target) & (target != ignore_index)  # Identify error positions
    errors_np = errors.cpu().numpy()  # Convert to NumPy for visualization
    #print("Errors:", errors_np)

    return loss.item(), accuracy

def fix_attn_pattern_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    attn_labels_1_idx,
    heads_to_reinforce
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    #value[:, :, :, :] += torch.tensor(neco, dtype=torch.float32).cuda()
    batch, heads, seq_len, _ = value.shape 
    
    layer = int(hook.name.split(".")[1])
    if layer not in heads_to_reinforce.keys():
        return
    for h in range(heads):
        if h != heads_to_reinforce[layer]:
            continue
        #visualize_attention_heads(value, None, title="Example Attention Heatmap", heads=[heads_to_reinforce[layer]])
        for i in range(seq_len):
            for j in range(seq_len):
                if (i, j) in attn_labels_1_idx:
                    value[0, h, i, j] += 2.1
                    pass
        #print("After corrections")
        #visualize_attention_heads(value, None, title="Example Attention Heatmap", heads=[heads_to_reinforce[layer]])

    return value

def inspect_rope_hook(
        value: Float[torch.Tensor, "batch pos head_index d_head"],
        hook: HookPoint,
        heads_to_reinforce
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    batch, heads, seq_len, _ = value.shape 
    
    layer = int(hook.name.split(".")[1])
    if layer not in heads_to_reinforce.keys():
        return

    
    for h in range(heads):
        if h != heads_to_reinforce[layer]:
            continue
        print(hook.name, h)
        for i in range(seq_len):
            for j in range(seq_len):
                if (i, j) in attn_labels_1_idx:
                    #value[0, h, i, j] += 1
                    pass
        #print("After corrections")
        #visualize_attention_heads(value, None, title="Example Attention Heatmap", heads=[heads_to_reinforce[layer]])

    return value


device = "cuda"

is_attn_scores = lambda name: name.endswith("hook_pattern")
is_rot_q = lambda name: name.endswith("hook_rot_q")
is_rot_k = lambda name: name.endswith("hook_rot_k")

for idx in range(0, 10):

    model.to(device)
    input_ids[idx] = input_ids[idx].to(device)
    target_ids[idx] = target_ids[idx].to(device)
    attn_labels[idx] = attn_labels[idx].to(device)

    attn_labels_1_idx = set()
    for i in range(len(attn_labels[idx])):
        for j in range(len(attn_labels[idx][0])):
            if attn_labels[idx][i][j] == 1:
                attn_labels_1_idx.add((i, j))

    heads_to_reinforce=[
        (11, 14),
        (12, 13),
        (14, 11),


    ]
    import random
    import itertools
    cols_to_keep = ~np.all(attn_labels[idx].cpu().numpy() == -100, axis=0)
    ablated = []
    for i in range(3,4):
        for heads in itertools.combinations(heads_to_reinforce, i):
            print(heads)
            heads = dict(heads)
            
            logits = model.run_with_hooks(input_ids[idx], return_type="logits",
                                            fwd_hooks=[
                                                (is_attn_scores, partial(fix_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads))
                                            ])
            
            #logits, cache = model.run_with_cache(input_ids[idx], return_type="logits")
            loss, acc = compute_scores(logits, target_ids[idx])
            #ablated.append((heads, loss, acc))
            #del logits
            #torch.cuda.empty_cache()
    

Moving model to device:  cuda


KeyboardInterrupt: 

In [14]:
len2_avgacc = {}
length = 1
while True:
    data_generator = StringReversalGenerator(tokenizer=model.tokenizer,
                                        seed=0,
                                        use_few_shot=True, 
                                        use_instruction=True, 
                                        apply_chat_template=True,
                                        length=(length,length),
                                        )
    samples = itertools.islice(data_generator.generate_samples(), 10)

    device = "cuda"

    is_attn_scores = lambda name: name.endswith("hook_pattern")
    avg_acc = []
    samples = list(samples)
    texts = [sample[0] for sample in samples]
    input_ids = [sample[1] for sample in samples]
    target_ids = [sample[2] for sample in samples]
    attn_labels = [sample[3] for sample in samples]
    print(length)
    if len(input_ids[0]) > 2048:
        break
    for idx in range(0, 10):

        model.to(device)
        input_ids[idx] = input_ids[idx].to(device)
        target_ids[idx] = target_ids[idx].to(device)
        attn_labels[idx] = attn_labels[idx].to(device)

        attn_labels_1_idx = set()
        for i in range(len(attn_labels[idx])):
            for j in range(len(attn_labels[idx][0])):
                if attn_labels[idx][i][j] == 1:
                    attn_labels_1_idx.add((i, j))

        heads_to_reinforce=[
            (11, 14),
            (12, 13),
            (14, 11),


        ]
        import random
        import itertools
        cols_to_keep = ~np.all(attn_labels[idx].cpu().numpy() == -100, axis=0)
        heads = dict(heads_to_reinforce)
        
        logits = model.run_with_hooks(input_ids[idx], return_type="logits",
                                        fwd_hooks=[
                                            (is_attn_scores, partial(fix_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads))
                                        ])
        
        #logits, cache = model.run_with_cache(input_ids[idx], return_type="logits")
        loss, acc = compute_scores(logits, target_ids[idx])
        avg_acc.append(acc)
        del logits
        torch.cuda.empty_cache()
    len2_avgacc[length] = sum(avg_acc) / len(avg_acc)
    print(sum(avg_acc) / len(avg_acc))
    length *= 2
    
    

1
Moving model to device:  cuda
Loss: 0.002885762369260192, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 2.443786115691182e-06, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 0.00033921466092579067, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 1.311301275563892e-06, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 1.019229875964811e-05, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 1.072882923836005e-06, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 0.0004086616390850395, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 0.004492259118705988, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 2.2649737729807384e-06, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 0.00030061532743275166, Accuracy: 100.00%
1.0
2
Moving model to device:  cuda
Loss: 0.029678253456950188, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 0.02724243886768818, Accuracy: 100.00%
Moving model to device:  cuda
Loss: 4.283315138309263e-05, Accuracy: 1

KeyboardInterrupt: 